In [1]:
import numpy as np
import tensorflow as tf
import os
import time

In [2]:
feat_dim = 120
fps = 100

x = tf.placeholder(tf.float32, shape = [None, feat_dim, fps])
y = tf.placeholder( tf.float32, shape = [None, 2])

input_x = tf.reshape( x, shape = [-1, feat_dim, fps, 1])

In [3]:
conv1 = tf.layers.conv2d( input_x, 
                                filters = 32,
                                kernel_size = 9,
                                padding = 'VALID',
                                activation = tf.nn.relu,
                                name = "conv1_caps")

# conv2 = tf.layers.conv2d( conv1, 
#                                 filters = 32,
#                                 kernel_size = 9,
#                                 padding = 'VALID',
#                                 activation = tf.nn.relu,
#                                 name = "conv2_caps")

conv_out = tf.layers.conv2d( conv1, 
                                filters = 32,
                                kernel_size = 9,
                                strides = 2,
                                padding = "VALID",
                                activation = tf.nn.relu,
                                name = "conv3_caps")

# conv4 = tf.layers.conv2d( conv3, 
#                                 filters = 64,
#                                 kernel_size = 9,
#                                 strides = 2,
#                                 padding = "VALID",
#                                 activation = tf.nn.relu,
#                                 name = "conv4_caps")
# print(conv3)
epsilon = 1e-7



In [4]:
conv_out

<tf.Tensor 'conv3_caps/Relu:0' shape=(?, 52, 42, 32) dtype=float32>

In [5]:
def squash(vector, axis = -2, keep_dims = True):
    norm = tf.reduce_sum( tf.square(vector), axis, keep_dims )
    scalar_factor = norm / (1. + norm)
    unit_vector = vector / tf.sqrt( norm + epsilon)
    return scalar_factor * unit_vector

In [6]:
#primary capsules
vector_dim = 8
total_capsules = int( conv_out.shape[1]*conv_out.shape[2]*conv_out.shape[3] //vector_dim)  #6 * 6 * 32
raw_capsules = tf.reshape( conv_out, shape = [-1, total_capsules, vector_dim ])
primary_capsules = squash( raw_capsules, -1, True)

In [7]:
print(primary_capsules)
batch_size = tf.shape(x)[0]

Tensor("mul:0", shape=(?, 8736, 8), dtype=float32)


In [8]:
sel_cap_dim = 512

fully_con_weights = tf.Variable( tf.random_normal( shape = [1, total_capsules, sel_cap_dim ], 
                                                 stddev = 0.1,
                                                 dtype = tf.float32))

fully_con_weights_tile = tf.tile( fully_con_weights, [batch_size, 1, 1])

selected_capusles = tf.matmul(primary_capsules, fully_con_weights_tile, transpose_a = True)
selected_capusles = tf.reshape(selected_capusles, shape = [-1, sel_cap_dim, vector_dim])
print(selected_capusles)

Tensor("Reshape_2:0", shape=(?, 512, 8), dtype=float32)


In [9]:
output_vector = 16
output_labels = 2
                                                            # [ 1, 1152, 10, 16, 8]
transfer_weights = tf.Variable( 
                        tf.random_normal( shape = [1, sel_cap_dim, output_labels, output_vector, vector_dim],
                                            stddev = 0.1, 
                                            dtype = tf.float32), 
                        name = "transfer_weights")

batch_size = tf.shape(x)[0]
tranf_weig_tiled = tf.tile( transfer_weights, [ batch_size, 1, 1, 1, 1], name ="tranf_weig_tiled" )
tranf_weig_tiled

<tf.Tensor 'tranf_weig_tiled:0' shape=(?, 512, 2, 16, 8) dtype=float32>

In [10]:
sel_caps_reshape = tf.reshape( selected_capusles, 
                              shape = [batch_size, sel_cap_dim, 1, vector_dim, 1],
                              name = "pri_caps_reshape")
sel_caps_tiled = tf.tile( sel_caps_reshape,
                         [1, 1, output_labels, 1, 1],
                         name = "pri_caps_tiled")


In [11]:
all_caps_pred_vect = tf.matmul( tranf_weig_tiled, sel_caps_tiled,
                              name = "raw_predictions")
all_caps_pred_vect

<tf.Tensor 'raw_predictions:0' shape=(?, 512, 2, 16, 1) dtype=float32>

In [12]:
# dynamic routing round-1
b = tf.zeros( shape = [1, sel_cap_dim, output_labels, 1, 1], 
             dtype = tf.float32,
             name = "b")

coup_coeff = tf.nn.softmax( b, 
                           dim = 2,
                           name = "coup_coeff")

weighted_all_cap_pred = tf.multiply( coup_coeff, all_caps_pred_vect)



Instructions for updating:
dim is deprecated, use axis instead


In [9]:
#sum over all 1152 capsules
weighted_pred_vec_r1 = tf.reduce_sum( weighted_all_cap_pred, 
                             axis = 1, 
                             keepdims = True,
                             name = "weighted_pred_vec_r1")

pred_vector_r1 = squash(weighted_pred_vec_r1)

pred_vector_r1


NameError: name 'weighted_all_cap_pred' is not defined

In [14]:
#agreements
pred_vect_r1_tile = tf.tile( pred_vector_r1,
                            [ 1, sel_cap_dim, 1, 1, 1],
                            name = "r1_tiled")
agrements = tf.matmul( all_caps_pred_vect, pred_vect_r1_tile ,
                     transpose_a = True,
                     name = 'agrement')
print(agrements)

Tensor("agrement:0", shape=(?, 512, 2, 1, 1), dtype=float32)


In [15]:

# round - 2

b = b + agrements
coup_coeff_r2 = tf.nn.softmax( b, dim = 2)
weighted_all_pred_vect_r2 = tf.multiply( coup_coeff_r2, all_caps_pred_vect)
weighted_pred_vect_r2 = tf.reduce_sum( weighted_all_pred_vect_r2, axis = 1, keep_dims = True)
pred_vect_r2 = squash(weighted_pred_vect_r2)


pred_vect_r2


predicted_vector = pred_vect_r2


sum_squares = tf.reduce_sum( tf.square(predicted_vector), axis =-2, keep_dims = False, name = "sum_squares" )
safe_norm1 = tf.sqrt( sum_squares + 1e-7)
print(safe_norm1)


# In[54]:


pred_prob = tf.reshape( safe_norm1, shape = [batch_size, 2])
pred_labels = tf.argmax( pred_prob, axis = 1)
print(pred_labels)


# In[58]:


# loss

m_plus = 0.9
m_minus = 0.1
lamda = 0.5

present_error_raw = tf.square( tf.maximum(0., m_plus - safe_norm1))
present_error = tf.reshape( present_error_raw, shape = [-1, 2])
print(present_error)

absent_error_raw = tf.square( tf.maximum(0., safe_norm1 - m_minus))
absent_error = tf.reshape( absent_error_raw, shape = [-1, 2])
print(absent_error)


# In[59]:


#T = tf.one_hot( y, depth = output_labels , name = "T")
T = y


# In[60]:


L = tf.add( T * present_error , (lamda * ((1. - T) * absent_error)), name = "L" )


# In[61]:


margin_loss = tf.reduce_mean( tf.reduce_sum(L, axis = 1), name ="margin_loss" )


# In[62]:


optimizer = tf.train.AdamOptimizer()
train_step = optimizer.minimize( margin_loss , name="train_step")


# correct_predictions = tf.equal( y, pred_labels , name="correct")
# accuracy = tf.reduce_mean( tf.cast(correct_predictions, tf.float32))

# In[ ]:

Tensor("Sqrt_3:0", shape=(?, 1, 2, 1), dtype=float32)
Tensor("ArgMax:0", shape=(?,), dtype=int64)
Tensor("Reshape_4:0", shape=(?, 2), dtype=float32)
Tensor("Reshape_5:0", shape=(?, 2), dtype=float32)


In [16]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2


# In[ ]:


from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"
    indexs = (len(list(os.listdir(test_path))))//2
    
    for folders in range(0, indexs):
        features = np.load(os.path.join(test_path, "features"+str(folders)+".npy"))
        labels = np.load(os.path.join(test_path, "labels"+str(folders)+".npy"))
        patches = labels.shape[0]
        patch_prob = session.run(pred_prob, feed_dict = {x:features})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_geniue = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_geniue.append(scores[i])
            prob_genuine.append(probabilities[i])
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_geniue = np.array(scores_geniue)
    scores_spoof = np.array(scores_spoof)
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    a,b = cal_eer(scores_geniue, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
        
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_geniue, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [17]:
#initializing tensors using sessoion
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=None)
features_path = "/home/speechlab/ASV/siva/IFCC/overlap_data_100_50"
development_path = "/home/speechlab/ASV/siva/IFCC/dev_overlap_data_100_50"
#evaluation_path = "E:\\Hemlata_CSL\\MFCC\\CNN_input_testing_eval"
model_path = "/home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/"
    
dev_sp_path ="/home/speechlab/ASV/siva/IFCC/CapsuleNet/dev"
#eva_sp_path = "E:\\Hemlata_CSL\\MFCC\\4_conv\\eval"

In [18]:
indexs = (len(list(os.listdir(development_path))))//2
print(indexs)
features = np.load(os.path.join(development_path, "labels"+str(0)+".npy"))
features = np.load(os.path.join(development_path, "labels"+str(indexs-1)+".npy"))

29700


In [19]:
# In[ ]:

# time.sleep(18000)
print("start")
#Training and testing accuracy
session.run(init)

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
# if not os.path.exists(dev_sp_path):
#     os.makedirs(dev_sp_path)
    
batches = (len(list(os.listdir(features_path))))//2
    
for i in range(0,50):
    for j in range(1,batches+1):
        #print(j)
        filename = "batch_"+str(j)+".npy"
        labelsname= "label_"+str(j)+".npy"
        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))
        
        mini_batches = features.shape[0]//50 + 1
        
        for l in range(mini_batches):
            train_step.run(session = session,feed_dict={ x:features[ l*50 : (l+1)*50 ], y:labels[ l*50 : (l+1)*50 ] })
        
    save_path = saver.save(session, model_path +str(i))
    print("iteration  ",(i+1))
    
'''
    print("iteration  ",(i+1))
    eer_dev = get_EER(development_path, (i+1), dev_sp_path)
    print(" develop eer  ",eer_dev)
    print()
'''

start
INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/0 is not in all_model_checkpoint_paths. Manually adding it.
iteration   1
INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/1 is not in all_model_checkpoint_paths. Manually adding it.
iteration   2
INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/2 is not in all_model_checkpoint_paths. Manually adding it.
iteration   3
INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/3 is not in all_model_checkpoint_paths. Manually adding it.
iteration   4
INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/4 is not in all_model_checkpoint_paths. Manually adding it.
iteration   5
INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/5 is not in all_model_checkpoint_paths. Manually adding it.
iteration   6
INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/6 is not in all_model_checkpoint_paths. Manually adding it.
iteration   7
INFO:tensorflow:/home/speech

'\n    print("iteration  ",(i+1))\n    eer_dev = get_EER(development_path, (i+1), dev_sp_path)\n    print(" develop eer  ",eer_dev)\n    print()\n'

In [20]:
# In[ ]:


with tf.Session() as session:
    for i in range(0,50):
        saver.restore(session, model_path +str(i))
        print("iteration  ",(i+1))
        eer_dev = get_EER(development_path, (i+1), dev_sp_path)
        print(" develop eer  ",eer_dev)
        print()



INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/0
iteration   1
FRR   0.3087037037037037  FAR    0.30872427983539097    mid    -2.4318726872252228e-06
 develop eer   0.3087037037037037

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/1
iteration   2
FRR   0.5566666666666666  FAR    0.5567901234567901    mid    -7.2275882810354215
 develop eer   0.5566666666666666

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/2
iteration   3
FRR   0.5666666666666667  FAR    0.5666666666666667    mid    -7.211490561133697
 develop eer   0.5666666666666667

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/3
iteration   4
FRR   0.49944444444444447  FAR    0.53440329218107    mid    -8.059047716004507
 develop eer   0.49944444444444447

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/4
iteration   5

iteration   38
FRR   0.4861111111111111  FAR    0.5086008230452674    mid    -8.05904771387577
 develop eer   0.4861111111111111

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/38
iteration   39
FRR   0.4816666666666667  FAR    0.5041152263374485    mid    -8.05904771387577
 develop eer   0.4816666666666667

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/39
iteration   40
FRR   0.48074074074074075  FAR    0.4939506172839506    mid    -8.05904771387577
 develop eer   0.48074074074074075

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/40
iteration   41
FRR   0.46  FAR    0.5263786008230452    mid    -8.059047716004507
 develop eer   0.46

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/CapsuleNet/Models/41
iteration   42
FRR   0.4803703703703704  FAR    0.5078600823045267    mid    -8.05904771387577
 develop eer   0.4803703703703704

INF